In [1]:
from scipy.linalg import lstsq
import numpy as np

In [3]:
T_1_to_2 = np.array([
    [0.07143, -0.6589, 0.7488, -3], 
    [0.9446, 0.2857, 0.1613, 5],
    [-0.3202, 0.6958, 0.6429, 7], 
    [0, 0, 0, 1]
])

In [210]:
points_frame_1 = np.array([
    [1, 3, 5], 
    [2, 4, 3], 
    [3, 7, 2],
    [-2, -1, 0], 
    [2, 9, 8],
    [4, 1, 6], 
    [1, 2, 2]
])
# points_frame_1 = np.random.rand(100, 3) * 5

In [211]:
points_frame_2 = np.pad(points_frame_1, [(0, 0), (0, 1)], 'constant', constant_values=1) @ T_1_to_2.T
noise = np.random.normal(loc=0.0, scale=0.3, size=points_frame_2.shape)
noise[:, 3] = 0
points_frame_2 += noise

In [185]:
M = np.pad(points_frame_1, [(0, 0), (0, 1)], 'constant', constant_values=1)

In [186]:
lstsq(M, points_frame_2[:])[0].T[:3]

array([[ 0.0488828 , -0.65970078,  0.78350535, -3.0106122 ],
       [ 0.97694906,  0.30755055,  0.18333153,  4.79231095],
       [-0.32811975,  0.68859159,  0.65074587,  7.00164291]])

In [212]:
points_frame_2_ = points_frame_2[:, :3]

In [213]:
centroid_frame_1 = np.mean(points_frame_1, axis=0)
centroid_frame_2 = np.mean(points_frame_2_, axis=0)
H = (points_frame_1 - centroid_frame_1).T @ (points_frame_2_ - centroid_frame_2)
U, S, V = np.linalg.svd(H)
R = (U @ V).T

In [214]:
T = centroid_frame_2 - R @ centroid_frame_1

In [215]:
R, T

(array([[ 0.07145326, -0.65892883,  0.74880386],
        [ 0.94464884,  0.28570803,  0.16127458],
        [-0.32020775,  0.69583311,  0.64287113]]),
 array([-2.99994793,  4.99998899,  7.00000118]))

In [249]:
def optimize_transformation(xyzs_camera, xyzs_robot):
    """
    args: 
      xyzs_camera = (N, 3) numpy array, containing points in camera frame
      xyzs_robot  = (N, 3) numpy array, containing points in robot frame
    returns:
      T = [4, 4] including a transformation matrix converting camera frame to robot frame
        = [[R(3x3) , t(3x1)],
           [0, 0, 0,      1]]
    """
    centroid_frame_camera = np.mean(points_frame_1, axis=0)
    centroid_frame_robot  = np.mean(points_frame_2_, axis=0)
    H = (points_frame_1 - centroid_frame_1).T @ (points_frame_2_ - centroid_frame_2)
    U, S, V = np.linalg.svd(H)
    R = (U @ V).T
    t = centroid_frame_2 - R @ centroid_frame_1
    return np.hstack([np.vstack([R.T, t]), np.array([[0, 0, 0, 1]]).T]).T

In [250]:
optimize_transformation(points_frame_1, points_frame_2_)

array([[ 0.07145326, -0.65892883,  0.74880386, -2.99994793],
       [ 0.94464884,  0.28570803,  0.16127458,  4.99998899],
       [-0.32020775,  0.69583311,  0.64287113,  7.00000118],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [259]:
def check_transformation_matrix(optimize_transformation, num_points):
    T_1_to_2 = np.array([
        [0.07143, -0.6589, 0.7488, -3], 
        [0.9446, 0.2857, 0.1613, 5],
        [-0.3202, 0.6958, 0.6429, 7], 
        [0, 0, 0, 1]
    ])
    
    points_frame_1 = np.random.rand(num_points, 3) * 5
    print(points_frame_1)
    points_frame_2 = np.pad(points_frame_1, [(0, 0), (0, 1)], 'constant', constant_values=1) @ T_1_to_2.T
    noise = np.random.normal(loc=0.0, scale=0.3, size=points_frame_2.shape)
    noise[:, 3] = 0
    points_frame_2_noise = points_frame_2 + noise
    
    points_frame_2_ = points_frame_2[:, :3]
    points_frame_2_noise_ = points_frame_2_noise[:, :3]
    
    T_calculated = optimize_transformation(points_frame_1, points_frame_2_noise_)
    points_frame_2_est = np.pad(points_frame_1, [(0, 0), (0, 1)], 'constant', constant_values=1) @ T_calculated.T
    points_frame_2_est = points_frame_2_est[:, :3]
    
    print(f'Original T: \n{T_1_to_2}')
    print(f'Calculated T: \n{T_calculated}')
    print(f'|T[Orig] - T[Calc]| = {np.linalg.norm(T_calculated - T_1_to_2)}\n')
    print(f'Average(T[calc].P - T[orig].P) = {np.mean(points_frame_2_est - points_frame_2_, axis=0)}')

In [260]:
check_transformation_matrix(optimize_transformation, 2)

[[3.30241271 2.3838876  2.5101413 ]
 [4.86175364 2.0720043  4.45137588]]
Original T: 
[[ 0.07143 -0.6589   0.7488  -3.     ]
 [ 0.9446   0.2857   0.1613   5.     ]
 [-0.3202   0.6958   0.6429   7.     ]
 [ 0.       0.       0.       1.     ]]
Calculated T: 
[[ 0.07145326 -0.65892883  0.74880386 -2.99994793]
 [ 0.94464884  0.28570803  0.16127458  4.99998899]
 [-0.32020775  0.69583311  0.64287113  7.00000118]
 [ 0.          0.          0.          1.        ]]
|T[Orig] - T[Calc]| = 9.64750369530397e-05

Average(T[calc].P - T[orig].P) = [ 9.62194644e-05  1.17762699e-04 -5.71901464e-05]
